# 🔬 Diagrammes Scientifiques avec GenAI

**Module :** Examples Sectoriels - Sciences  
**Niveau :** 🟡 Intermédiaire  
**Technologies :** DALL-E 3, GPT-5 Vision, OpenRouter  
**Durée estimée :** 35 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Générer des diagrammes biologiques (cellules, systèmes, anatomie)
- [ ] Créer des illustrations physique/chimie (molécules, circuits, ondes)
- [ ] Produire des graphiques mathématiques (fonctions, géométrie)
- [ ] Concevoir des schémas techniques annotés
- [ ] Optimiser les prompts pour précision scientifique
- [ ] Valider l'exactitude éducative des images générées

## 📚 Prérequis

- Notebooks Foundation (01-1, 01-2) complétés
- Compréhension des concepts scientifiques de base
- Familiarité avec le prompt engineering

In [1]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"               

# Configuration génération
science_domain = "biology"         # "biology", "physics", "chemistry", "math"
diagram_style = "educational"      # "educational", "textbook", "poster", "technical"
image_quality = "hd"               # "standard" ou "hd"
include_labels = True              # Ajouter labels et annotations
generate_variations = True         # Créer variations par sujet

# Sujets scientifiques
biology_topics = ["cell_structure", "photosynthesis", "dna_replication"]
physics_topics = ["circuit_diagram", "wave_interference", "forces"]
chemistry_topics = ["molecular_structure", "periodic_table", "reactions"]
math_topics = ["quadratic_functions", "geometric_shapes", "probability"]

# Sauvegarde et export
save_diagrams = True               # Sauvegarder images générées
export_lesson_pack = True          # Créer pack pédagogique complet

In [2]:
from dotenv import load_dotenv
load_dotenv("../../.env")

# Setup environnement et imports
import os
import sys
import json
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import base64
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import logging

# Configuration paths
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'science_diagrams'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('science_diagrams')

# Configuration API
openrouter_key = os.getenv('OPENROUTER_API_KEY')
if not openrouter_key:
    raise ValueError("❌ OPENROUTER_API_KEY manquante")

api_base_url = "https://openrouter.ai/api/v1"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "HTTP-Referer": "https://coursia.myia.io",
    "X-Title": "CoursIA Science Diagrams",
    "Content-Type": "application/json"
}

print(f"🔬 Diagrammes Scientifiques avec GenAI")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Domaine: {science_domain}, Style: {diagram_style}")
print(f"📁 Output: {OUTPUT_DIR}")
print(f"✅ API configurée")

🔬 Diagrammes Scientifiques avec GenAI
📅 2026-02-25 20:57:25
🔧 Domaine: biology, Style: educational
📁 Output: D:\Dev\CoursIA.worktrees\GenAI_Series\MyIA.AI.Notebooks\GenAI\outputs\science_diagrams
✅ API configurée


## 🧬 Section 1: Diagrammes Biologiques

Génération de diagrammes pour concepts biologiques : cellules, systèmes, anatomie.

In [3]:
# Templates de prompts biologiques optimisés
biology_prompt_templates = {
    "cell_structure": {
        "title": "🔬 Structure Cellulaire",
        "prompt": """Create a detailed educational diagram of an animal cell cross-section showing:
- Cell membrane (outer boundary)
- Nucleus with nuclear envelope and chromatin
- Mitochondria (powerhouse)
- Endoplasmic reticulum (rough and smooth)
- Golgi apparatus
- Ribosomes
- Lysosomes
- Cytoplasm

Style: Clean textbook illustration with clear labels pointing to each organelle. 
Use soft colors: blue for nucleus, green for mitochondria, pink for ER. 
White background. Professional biology textbook quality. Highly detailed but not overwhelming.""",
        "learning_objective": "Identifier les organites cellulaires et leurs fonctions"
    },
    "photosynthesis": {
        "title": "🌱 Processus de Photosynthèse",
        "prompt": """Create an educational diagram showing the photosynthesis process in a plant leaf:
- Sunlight arrows entering the leaf
- Chloroplast with thylakoid stacks
- CO2 entering through stomata
- H2O from roots
- Light-dependent reactions in thylakoids
- Calvin cycle in stroma
- O2 release
- Glucose production

Style: Simplified but scientifically accurate. Use green for chlorophyll, yellow for light energy, 
blue for water, red for oxygen. Clear arrows showing flow. Educational poster style.""",
        "learning_objective": "Comprendre les étapes de la photosynthèse"
    },
    "dna_replication": {
        "title": "🧬 Réplication de l'ADN",
        "prompt": """Create an educational diagram of DNA replication showing:
- Double helix unwinding
- Helicase enzyme separating strands
- DNA polymerase adding nucleotides
- Leading strand (continuous synthesis)
- Lagging strand (Okazaki fragments)
- RNA primers
- Ligase joining fragments

Style: Scientific textbook illustration. Use classic DNA double helix colors (blue and red for strands),
yellow for enzymes. Clear labels. Show directionality (5' to 3'). Clean white background.""",
        "learning_objective": "Maîtriser le mécanisme de réplication de l'ADN"
    },
    "human_heart": {
        "title": "❤️ Anatomie du Cœur Humain",
        "prompt": """Create a detailed anatomical diagram of the human heart showing:
- Four chambers: right/left atria and ventricles
- Valves: tricuspid, pulmonary, mitral, aortic
- Major blood vessels: aorta, vena cava, pulmonary arteries/veins
- Blood flow direction with arrows (blue for deoxygenated, red for oxygenated)
- Heart walls and septum

Style: Medical textbook quality. Use red for oxygenated blood, blue for deoxygenated. 
Clear labels. Cross-sectional view. Professional anatomical illustration.""",
        "learning_objective": "Comprendre l'anatomie et la circulation cardiaque"
    }
}

print("\n🧬 DIAGRAMMES BIOLOGIQUES")
print("=" * 50)
print(f"\n📚 {len(biology_prompt_templates)} templates biologiques disponibles:")
for key, template in biology_prompt_templates.items():
    print(f"\n{template['title']}")
    print(f"   🎯 Objectif: {template['learning_objective']}")
    print(f"   📝 Prompt: {len(template['prompt'])} caractères")


🧬 DIAGRAMMES BIOLOGIQUES

📚 4 templates biologiques disponibles:

🔬 Structure Cellulaire
   🎯 Objectif: Identifier les organites cellulaires et leurs fonctions
   📝 Prompt: 533 caractères

🌱 Processus de Photosynthèse
   🎯 Objectif: Comprendre les étapes de la photosynthèse
   📝 Prompt: 488 caractères

🧬 Réplication de l'ADN
   🎯 Objectif: Maîtriser le mécanisme de réplication de l'ADN
   📝 Prompt: 469 caractères

❤️ Anatomie du Cœur Humain
   🎯 Objectif: Comprendre l'anatomie et la circulation cardiaque
   📝 Prompt: 492 caractères


In [4]:
# Fonction de génération avec validation
def generate_science_diagram(prompt: str,
                           title: str,
                           category: str,
                           quality: str = "hd") -> Dict[str, Any]:
    """
    Génère un diagramme scientifique avec DALL-E 3.
    
    Args:
        prompt: Description détaillée du diagramme
        title: Titre du diagramme
        category: Catégorie (biology, physics, etc.)
        quality: Qualité de l'image
    
    Returns:
        Dict avec résultats et métadonnées
    """
    payload = {
        "model": "openai/dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1792x1024",  # Format large pour diagrammes
        "quality": quality,
        "style": "natural",  # Style naturel pour diagrammes scientifiques
        "response_format": "url"
    }
    
    try:
        print(f"\n🎨 Génération: {title}")
        print(f"   📊 Catégorie: {category}")
        print(f"   ⚙️ Qualité: {quality}")
        
        response = requests.post(
            f"{api_base_url}/images/generations",
            headers=headers,
            json=payload,
            timeout=90
        )
        
        if response.status_code == 200:
            result = response.json()
            image_url = result["data"][0]["url"]
            revised_prompt = result["data"][0].get("revised_prompt", prompt)
            
            print(f"   ✅ Généré en {response.elapsed.total_seconds():.1f}s")
            
            return {
                "success": True,
                "image_url": image_url,
                "title": title,
                "category": category,
                "original_prompt": prompt,
                "revised_prompt": revised_prompt,
                "timestamp": datetime.now().isoformat(),
                "quality": quality
            }
        else:
            error = response.json().get("error", {}).get("message", "Unknown")
            print(f"   ❌ Erreur: {error}")
            return {"success": False, "error": error}
            
    except Exception as e:
        print(f"   ❌ Exception: {str(e)[:100]}")
        return {"success": False, "error": str(e)}

def download_and_display(result: Dict[str, Any], save: bool = True) -> Optional[Image.Image]:
    """
    Télécharge et affiche un diagramme généré.
    """
    if not result.get('success'):
        return None
    
    try:
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            # Affichage
            plt.figure(figsize=(16, 9))
            plt.imshow(image)
            plt.title(f"{result['title']} - {result['category'].upper()}", 
                     fontsize=14, fontweight='bold', pad=20)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            # Sauvegarde
            if save:
                category_dir = OUTPUT_DIR / result['category']
                category_dir.mkdir(exist_ok=True)
                
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                safe_title = result['title'].replace(' ', '_').replace('/', '_')[:50]
                filename = f"{safe_title}_{timestamp}.png"
                filepath = category_dir / filename
                
                image.save(filepath)
                print(f"   💾 Sauvegardé: {filepath.name}")
                
                # Métadonnées
                meta_file = category_dir / f"{safe_title}_{timestamp}_meta.json"
                with open(meta_file, 'w', encoding='utf-8') as f:
                    json.dump(result, f, indent=2, ensure_ascii=False)
            
            return image
    except Exception as e:
        print(f"   ❌ Erreur affichage: {str(e)[:100]}")
    
    return None

print("\n✅ Fonctions de génération prêtes")


✅ Fonctions de génération prêtes


In [5]:
# Génération: Structure Cellulaire
if science_domain in ["biology", "all"]:
    print("\n" + "=" * 60)
    print("🔬 GÉNÉRATION: STRUCTURE CELLULAIRE")
    print("=" * 60)
    
    cell_template = biology_prompt_templates["cell_structure"]
    
    cell_result = generate_science_diagram(
        prompt=cell_template["prompt"],
        title=cell_template["title"],
        category="biology",
        quality=image_quality
    )
    
    if cell_result['success']:
        print(f"\n📝 Prompt optimisé par DALL-E 3:")
        print(f"   {cell_result['revised_prompt'][:200]}...")
        
        cell_image = download_and_display(cell_result, save=save_diagrams)
        
        print(f"\n✅ Diagramme cellulaire généré avec succès")
        print(f"🎯 Objectif: {cell_template['learning_objective']}")
        print(f"\n💡 Utilisations pédagogiques:")
        print(f"   • Cours de biologie cellulaire niveau lycée")
        print(f"   • Support visuel pour identification des organites")
        print(f"   • Base pour exercices d'annotation")
        print(f"   • Comparaison cellule animale vs végétale")
else:
    print(f"\n⏭️  Domaine biology non sélectionné (science_domain={science_domain})")


🔬 GÉNÉRATION: STRUCTURE CELLULAIRE

🎨 Génération: 🔬 Structure Cellulaire
   📊 Catégorie: biology
   ⚙️ Qualité: hd


   ❌ Exception: Expecting value: line 1 column 1 (char 0)


## ⚛️ Section 2: Illustrations Physique & Chimie

Circuits électriques, structures moléculaires, phénomènes physiques.

In [6]:
# Templates physique/chimie
physics_chemistry_templates = {
    "circuit_diagram": {
        "title": "⚡ Schéma de Circuit Électrique",
        "prompt": """Create an educational electrical circuit diagram showing:
- Battery (power source) with + and - terminals
- Switch (open and closed states)
- Resistors (2-3 in series and parallel)
- LED or light bulb
- Ammeter measuring current
- Voltmeter measuring voltage
- Clear wire connections
- Current flow direction arrows

Style: Standard electrical engineering symbols. Clean, technical drawing. 
Black lines on white background. Clear labels for each component. 
Show values (e.g., 9V battery, 100Ω resistor). Professional schematic style.""",
        "learning_objective": "Comprendre les circuits électriques et la loi d'Ohm"
    },
    "molecular_structure": {
        "title": "🧪 Structure Moléculaire (Eau H2O)",
        "prompt": """Create an educational molecular structure diagram of water (H2O) showing:
- Central oxygen atom (red sphere)
- Two hydrogen atoms (small white spheres)
- Covalent bonds (solid lines)
- Bond angle (104.5 degrees)
- Partial charges (δ+ and δ-)
- Electron pairs (lone pairs on oxygen)
- 3D ball-and-stick model

Style: Chemistry textbook illustration. Use standard CPK coloring (red oxygen, white hydrogen).
Clean white background. Show molecular geometry. Include labels and measurements.""",
        "learning_objective": "Visualiser la géométrie et la polarité moléculaire"
    },
    "wave_interference": {
        "title": "🌊 Interférence d'Ondes",
        "prompt": """Create an educational physics diagram showing wave interference:
- Two wave sources creating circular ripples
- Constructive interference regions (bright/high amplitude)
- Destructive interference regions (dark/low amplitude)
- Wave crests and troughs clearly visible
- Pattern of nodes and antinodes
- Distance measurements (wavelength λ)
- Color coding: blue for constructive, red for destructive

Style: Clear physics textbook illustration. Top view of water surface. 
Smooth gradients showing interference pattern. Labels and annotations.""",
        "learning_objective": "Comprendre les phénomènes d'interférence ondulatoire"
    },
    "forces_diagram": {
        "title": "💪 Diagramme de Forces",
        "prompt": """Create an educational free body diagram showing forces on an object:
- Box or sphere on an inclined plane
- Weight force (mg) pointing downward (red arrow)
- Normal force perpendicular to surface (blue arrow)
- Friction force along surface (green arrow)
- Applied force if any (yellow arrow)
- Angle of incline marked
- Force vectors with magnitude labels
- Coordinate system (x, y axes)

Style: Clean physics diagram. Bold colored arrows for forces. 
White background. Clear labels with force equations. Professional textbook quality.""",
        "learning_objective": "Analyser les forces et le mouvement sur plan incliné"
    }
}

print("\n⚛️ PHYSIQUE & CHIMIE")
print("=" * 50)
print(f"\n📚 {len(physics_chemistry_templates)} templates disponibles:")
for key, template in physics_chemistry_templates.items():
    print(f"\n{template['title']}")
    print(f"   🎯 {template['learning_objective']}")


⚛️ PHYSIQUE & CHIMIE

📚 4 templates disponibles:

⚡ Schéma de Circuit Électrique
   🎯 Comprendre les circuits électriques et la loi d'Ohm

🧪 Structure Moléculaire (Eau H2O)
   🎯 Visualiser la géométrie et la polarité moléculaire

🌊 Interférence d'Ondes
   🎯 Comprendre les phénomènes d'interférence ondulatoire

💪 Diagramme de Forces
   🎯 Analyser les forces et le mouvement sur plan incliné


In [7]:
# Génération: Circuit Électrique
if science_domain in ["physics", "all"]:
    print("\n" + "=" * 60)
    print("⚡ GÉNÉRATION: CIRCUIT ÉLECTRIQUE")
    print("=" * 60)
    
    circuit_template = physics_chemistry_templates["circuit_diagram"]
    
    circuit_result = generate_science_diagram(
        prompt=circuit_template["prompt"],
        title=circuit_template["title"],
        category="physics",
        quality=image_quality
    )
    
    if circuit_result['success']:
        circuit_image = download_and_display(circuit_result, save=save_diagrams)
        
        print(f"\n✅ Schéma de circuit généré")
        print(f"\n💡 Applications pédagogiques:")
        print(f"   • Cours de physique électricité")
        print(f"   • TP circuits séries et parallèles")
        print(f"   • Calculs loi d'Ohm")
        print(f"   • Exercices de schématisation")
else:
    print(f"\n⏭️  Domaine physics non sélectionné")

# Génération: Structure Moléculaire
if science_domain in ["chemistry", "all"]:
    print("\n" + "=" * 60)
    print("🧪 GÉNÉRATION: STRUCTURE MOLÉCULAIRE")
    print("=" * 60)
    
    molecule_template = physics_chemistry_templates["molecular_structure"]
    
    molecule_result = generate_science_diagram(
        prompt=molecule_template["prompt"],
        title=molecule_template["title"],
        category="chemistry",
        quality=image_quality
    )
    
    if molecule_result['success']:
        molecule_image = download_and_display(molecule_result, save=save_diagrams)
        
        print(f"\n✅ Structure moléculaire générée")
        print(f"\n💡 Applications pédagogiques:")
        print(f"   • Cours de chimie liaisons")
        print(f"   • Géométrie moléculaire VSEPR")
        print(f"   • Polarité et liaisons hydrogène")
        print(f"   • Propriétés de l'eau")
else:
    print(f"\n⏭️  Domaine chemistry non sélectionné")


⏭️  Domaine physics non sélectionné

⏭️  Domaine chemistry non sélectionné


## 📐 Section 3: Graphiques Mathématiques

Fonctions, géométrie, probabilités - visualisations mathématiques.

In [8]:
# Templates mathématiques
math_templates = {
    "quadratic_function": {
        "title": "📈 Fonction Quadratique (Parabole)",
        "prompt": """Create an educational graph of a quadratic function showing:
- Coordinate system with x and y axes clearly labeled
- Parabola curve (y = ax² + bx + c)
- Vertex point clearly marked with coordinates
- X-intercepts (roots) marked
- Y-intercept marked
- Axis of symmetry (dashed vertical line)
- Grid lines for reading values
- Key points labeled with coordinates

Style: Clean mathematical graph. Use classic colors: blue curve, red points. 
White background with light gray grid. Professional textbook quality. 
Clear, readable labels and numbers.""",
        "learning_objective": "Comprendre les propriétés des fonctions quadratiques"
    },
    "geometric_shapes": {
        "title": "📐 Théorème de Pythagore",
        "prompt": """Create an educational geometric diagram illustrating the Pythagorean theorem:
- Right triangle with sides a, b, and hypotenuse c clearly labeled
- Right angle marked with small square
- Squares drawn on each side showing areas
- Area of each square labeled (a², b², c²)
- Visual proof that a² + b² = c²
- Clean colors: different color for each square
- Measurements shown (e.g., a=3, b=4, c=5)

Style: Geometric diagram, educational poster style. Bright, clear colors. 
White background. Professional mathematical illustration.""",
        "learning_objective": "Visualiser et comprendre le théorème de Pythagore"
    },
    "probability_tree": {
        "title": "🎲 Arbre de Probabilités",
        "prompt": """Create an educational probability tree diagram showing:
- Two coin flips (or dice rolls)
- Clear branches for each outcome
- Probabilities labeled on each branch (fractions or decimals)
- All possible outcomes at the end
- Total probability for each final outcome
- Clean structure: vertical or horizontal tree
- Color coding for different paths

Style: Educational diagram. Use simple, clear graphics. 
Circles or boxes for nodes. Straight lines for branches. 
Clear labels. Professional statistics textbook style.""",
        "learning_objective": "Comprendre les probabilités composées"
    },
    "3d_geometry": {
        "title": "📦 Géométrie 3D (Solides)",
        "prompt": """Create an educational 3D geometry diagram showing common solids:
- Cube with visible edges and faces
- Rectangular prism (box)
- Cylinder
- Cone
- Sphere
- Pyramid
Each shape labeled with:
- Name
- Key measurements (length, width, height, radius)
- Volume formula
- Surface area formula (if space permits)

Style: Isometric or perspective view. Soft shading to show 3D depth. 
Pastel colors for different shapes. White background. Clear labels. 
Educational geometry textbook quality.""",
        "learning_objective": "Reconnaître et calculer volumes des solides"
    }
}

print("\n📐 MATHÉMATIQUES")
print("=" * 50)
print(f"\n📚 {len(math_templates)} templates mathématiques:")
for key, template in math_templates.items():
    print(f"\n{template['title']}")
    print(f"   🎯 {template['learning_objective']}")


📐 MATHÉMATIQUES

📚 4 templates mathématiques:

📈 Fonction Quadratique (Parabole)
   🎯 Comprendre les propriétés des fonctions quadratiques

📐 Théorème de Pythagore
   🎯 Visualiser et comprendre le théorème de Pythagore

🎲 Arbre de Probabilités
   🎯 Comprendre les probabilités composées

📦 Géométrie 3D (Solides)
   🎯 Reconnaître et calculer volumes des solides


In [9]:
# Génération: Fonction Quadratique
if science_domain in ["math", "all"]:
    print("\n" + "=" * 60)
    print("📈 GÉNÉRATION: FONCTION QUADRATIQUE")
    print("=" * 60)
    
    quadratic_template = math_templates["quadratic_function"]
    
    quadratic_result = generate_science_diagram(
        prompt=quadratic_template["prompt"],
        title=quadratic_template["title"],
        category="math",
        quality=image_quality
    )
    
    if quadratic_result['success']:
        quadratic_image = download_and_display(quadratic_result, save=save_diagrams)
        
        print(f"\n✅ Graphique quadratique généré")
        print(f"\n💡 Applications pédagogiques:")
        print(f"   • Cours de mathématiques seconde/première")
        print(f"   • Étude de fonctions")
        print(f"   • Résolution équations du second degré")
        print(f"   • Exercices de lecture graphique")

# Génération: Théorème de Pythagore
if science_domain in ["math", "all"]:
    print("\n" + "=" * 60)
    print("📐 GÉNÉRATION: THÉORÈME DE PYTHAGORE")
    print("=" * 60)
    
    pythagoras_template = math_templates["geometric_shapes"]
    
    pythagoras_result = generate_science_diagram(
        prompt=pythagoras_template["prompt"],
        title=pythagoras_template["title"],
        category="math",
        quality=image_quality
    )
    
    if pythagoras_result['success']:
        pythagoras_image = download_and_display(pythagoras_result, save=save_diagrams)
        
        print(f"\n✅ Diagramme Pythagore généré")
        print(f"\n💡 Applications pédagogiques:")
        print(f"   • Cours de géométrie collège")
        print(f"   • Démonstration visuelle du théorème")
        print(f"   • Calculs de distances")
        print(f"   • Exercices d'application")
else:
    print(f"\n⏭️  Domaine math non sélectionné")


⏭️  Domaine math non sélectionné


## ✅ Section 4: Validation et Qualité Éducative

Vérification de l'exactitude scientifique avec GPT-5 Vision.

In [10]:
# Validation avec GPT-5 Vision
def validate_scientific_accuracy(image_url: str, 
                               diagram_type: str,
                               expected_content: str) -> Dict[str, Any]:
    """
    Valide l'exactitude scientifique d'un diagramme généré.
    
    Args:
        image_url: URL de l'image à valider
        diagram_type: Type de diagramme (cell, circuit, etc.)
        expected_content: Description du contenu attendu
    
    Returns:
        Dict avec évaluation et recommandations
    """
    validation_prompt = f"""Vous êtes un expert pédagogue en sciences. 
Analysez ce diagramme scientifique ({diagram_type}) et évaluez:

1. EXACTITUDE SCIENTIFIQUE (1-10):
   - Les concepts sont-ils représentés correctement?
   - Y a-t-il des erreurs factuelles?
   - Les proportions/échelles sont-elles réalistes?

2. CLARTÉ PÉDAGOGIQUE (1-10):
   - Le diagramme est-il facile à comprendre?
   - Les labels sont-ils clairs et lisibles?
   - Les couleurs aident-elles la compréhension?

3. COMPLÉTUDE:
   - Éléments attendus: {expected_content}
   - Quels éléments sont présents?
   - Quels éléments manquent?

4. RECOMMANDATIONS:
   - Améliorations suggérées
   - Points forts à conserver

Format de réponse: JSON avec scores et commentaires détaillés."""
    
    payload = {
        "model": "openai/gpt-4o",  # GPT-5 Vision via OpenRouter
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": validation_prompt},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }
        ],
        "max_tokens": 1000
    }
    
    try:
        response = requests.post(
            f"{api_base_url}/chat/completions",
            headers=headers,
            json=payload,
            timeout=60
        )
        
        if response.status_code == 200:
            result = response.json()
            analysis = result['choices'][0]['message']['content']
            
            return {
                "success": True,
                "analysis": analysis,
                "diagram_type": diagram_type,
                "timestamp": datetime.now().isoformat()
            }
        else:
            return {"success": False, "error": f"HTTP {response.status_code}"}
            
    except Exception as e:
        return {"success": False, "error": str(e)}

print("\n✅ VALIDATION DE LA QUALITÉ ÉDUCATIVE")
print("=" * 50)

# Validation du diagramme cellulaire (si généré)
if science_domain in ["biology", "all"] and 'cell_result' in locals() and cell_result.get('success'):
    print("\n🔍 Validation: Structure Cellulaire")
    
    validation = validate_scientific_accuracy(
        image_url=cell_result['image_url'],
        diagram_type="cell_structure",
        expected_content="organites: noyau, mitochondries, RE, Golgi, ribosomes, lysosomes"
    )
    
    if validation['success']:
        print(f"\n📊 Analyse GPT-5 Vision:")
        print(f"{validation['analysis']}")
        print(f"\n✅ Validation terminée")
    else:
        print(f"\n⚠️  Validation échouée: {validation.get('error')}")
else:
    print("\n💡 Validation disponible après génération des diagrammes")
    print("   Utilisez validate_scientific_accuracy() sur vos images")


✅ VALIDATION DE LA QUALITÉ ÉDUCATIVE

💡 Validation disponible après génération des diagrammes
   Utilisez validate_scientific_accuracy() sur vos images


## 📦 Section 5: Pack Pédagogique Complet

Création d'un pack de cours avec diagrammes, descriptions, et exercices.

In [11]:
if export_lesson_pack:
    print("\n📦 EXPORT PACK PÉDAGOGIQUE")
    print("=" * 50)
    
    # Création du pack
    pack_dir = OUTPUT_DIR / f"lesson_pack_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    pack_dir.mkdir(exist_ok=True)
    
    # Structure du pack
    (pack_dir / 'images').mkdir(exist_ok=True)
    (pack_dir / 'descriptions').mkdir(exist_ok=True)
    (pack_dir / 'exercises').mkdir(exist_ok=True)
    
    print(f"\n📁 Structure créée: {pack_dir.name}")
    
    # Index des ressources
    pack_index = {
        "title": f"Pack Pédagogique Sciences - {science_domain.upper()}",
        "created": datetime.now().isoformat(),
        "domain": science_domain,
        "style": diagram_style,
        "quality": image_quality,
        "resources": []
    }
    
    # Collecte des diagrammes générés
    generated_diagrams = []
    if 'cell_result' in locals() and cell_result.get('success'):
        generated_diagrams.append(('cell', cell_result, biology_prompt_templates['cell_structure']))
    if 'circuit_result' in locals() and circuit_result.get('success'):
        generated_diagrams.append(('circuit', circuit_result, physics_chemistry_templates['circuit_diagram']))
    if 'molecule_result' in locals() and molecule_result.get('success'):
        generated_diagrams.append(('molecule', molecule_result, physics_chemistry_templates['molecular_structure']))
    if 'quadratic_result' in locals() and quadratic_result.get('success'):
        generated_diagrams.append(('quadratic', quadratic_result, math_templates['quadratic_function']))
    if 'pythagoras_result' in locals() and pythagoras_result.get('success'):
        generated_diagrams.append(('pythagoras', pythagoras_result, math_templates['geometric_shapes']))
    
    print(f"\n📊 {len(generated_diagrams)} diagrammes à inclure")
    
    # Export de chaque diagramme avec métadonnées
    for key, result, template in generated_diagrams:
        print(f"\n📝 Export: {result['title']}")
        
        # Téléchargement image
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            # Sauvegarde image
            img_file = pack_dir / 'images' / f"{key}_diagram.png"
            image.save(img_file)
            print(f"   ✅ Image: {img_file.name}")
            
            # Description pédagogique
            description = f"""# {result['title']}

## 🎯 Objectif Pédagogique
{template['learning_objective']}

## 📋 Description
Ce diagramme illustre {result['category']} de manière claire et pédagogique.

## 🔍 Éléments Clés à Observer
(À compléter en fonction du contenu spécifique)

## 💡 Questions de Compréhension
1. Identifiez les principaux éléments du diagramme
2. Expliquez la fonction de chaque élément
3. Quelles sont les relations entre les éléments?

## 📚 Ressources Complémentaires
- Lien vers cours théorique
- Vidéos explicatives
- Exercices pratiques
"""
            desc_file = pack_dir / 'descriptions' / f"{key}_description.md"
            with open(desc_file, 'w', encoding='utf-8') as f:
                f.write(description)
            print(f"   ✅ Description: {desc_file.name}")
            
            # Ajout à l'index
            pack_index['resources'].append({
                "id": key,
                "title": result['title'],
                "category": result['category'],
                "image_file": f"images/{key}_diagram.png",
                "description_file": f"descriptions/{key}_description.md",
                "learning_objective": template['learning_objective']
            })
    
    # Export index
    index_file = pack_dir / 'README.md'
    readme_content = f"""# {pack_index['title']}

**Créé le:** {datetime.now().strftime('%Y-%m-%d %H:%M')}
**Domaine:** {science_domain}
**Qualité:** {image_quality}

## 📚 Contenu du Pack

"""
    for resource in pack_index['resources']:
        readme_content += f"""### {resource['title']}
- **Objectif:** {resource['learning_objective']}
- **Image:** `{resource['image_file']}`
- **Description:** `{resource['description_file']}`

"""
    
    with open(index_file, 'w', encoding='utf-8') as f:
        f.write(readme_content)
    
    # Export JSON
    json_file = pack_dir / 'pack_index.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(pack_index, f, indent=2, ensure_ascii=False)
    
    print(f"\n" + "=" * 50)
    print(f"✅ Pack pédagogique exporté: {pack_dir.name}")
    print(f"📁 {len(pack_index['resources'])} ressources incluses")
    print(f"📄 README.md créé")
    print(f"📊 pack_index.json créé")
    print(f"\n💡 Utilisez ce pack pour:")
    print(f"   • Préparation de cours")
    print(f"   • Supports de présentation")
    print(f"   • Exercices et évaluations")
    print(f"   • Ressources numériques élèves")
else:
    print("\n⏭️  Export pack désactivé (export_lesson_pack=False)")


📦 EXPORT PACK PÉDAGOGIQUE

📁 Structure créée: lesson_pack_20260225_205726

📊 0 diagrammes à inclure

✅ Pack pédagogique exporté: lesson_pack_20260225_205726
📁 0 ressources incluses
📄 README.md créé
📊 pack_index.json créé

💡 Utilisez ce pack pour:
   • Préparation de cours
   • Supports de présentation
   • Exercices et évaluations
   • Ressources numériques élèves


## 📚 Récapitulatif et Bonnes Pratiques

Consignes pour génération optimale de diagrammes scientifiques.

In [12]:
print("\n📚 RÉCAPITULATIF ET BONNES PRATIQUES")
print("=" * 50)

print("\n✅ Domaines Couverts:")
print("   🧬 Biologie: Cellules, systèmes, anatomie")
print("   ⚛️ Physique: Circuits, ondes, forces")
print("   🧪 Chimie: Molécules, réactions, structures")
print("   📐 Mathématiques: Graphiques, géométrie, probabilités")

print("\n💡 Bonnes Pratiques Prompt Engineering:")
print("\n1️⃣ Spécificité Scientifique:")
print("   • Nommer explicitement tous les éléments attendus")
print("   • Préciser les mesures, angles, proportions")
print("   • Utiliser la terminologie technique correcte")
print("   • Mentionner le niveau éducatif cible")

print("\n2️⃣ Style Visuel:")
print("   • 'textbook illustration' pour style académique")
print("   • 'clean white background' pour clarté")
print("   • 'clear labels' pour lisibilité")
print("   • Spécifier les couleurs standard (CPK, etc.)")

print("\n3️⃣ Pédagogie:")
print("   • 'educational diagram' oriente vers usage scolaire")
print("   • 'simplified but accurate' équilibre détail/clarté")
print("   • Demander annotations et légendes")
print("   • Inclure échelles et mesures")

print("\n4️⃣ Qualité Technique:")
print("   • Utiliser 'hd' pour diagrammes détaillés")
print("   • Format 1792x1024 optimal pour diagrammes")
print("   • Style 'natural' meilleur que 'vivid' pour science")
print("   • Toujours valider avec GPT-5 Vision")

print("\n⚠️  Points de Vigilance:")
print("   • Vérifier l'exactitude scientifique (IA peut se tromper)")
print("   • Confirmer que tous les éléments demandés sont présents")
print("   • Adapter le niveau de détail au public cible")
print("   • Tester la lisibilité sur différents supports")

print("\n🔗 Ressources Complémentaires:")
print("   • tutorials/dalle3-complete-guide.md")
print("   • tutorials/gpt5-image-analysis-guide.md")
print("   • tutorials/educational-workflows.md")

print("\n🚀 Prochaines Étapes:")
print("   1. Adapter les prompts à vos besoins spécifiques")
print("   2. Créer une bibliothèque de diagrammes")
print("   3. Valider avec des experts de domaine")
print("   4. Intégrer dans vos cours et supports")

print("\n" + "=" * 50)
print("✅ Notebook Science Diagrams terminé!")
print("🎓 Vous savez maintenant créer des diagrammes scientifiques de qualité")
print("=" * 50)


📚 RÉCAPITULATIF ET BONNES PRATIQUES

✅ Domaines Couverts:
   🧬 Biologie: Cellules, systèmes, anatomie
   ⚛️ Physique: Circuits, ondes, forces
   🧪 Chimie: Molécules, réactions, structures
   📐 Mathématiques: Graphiques, géométrie, probabilités

💡 Bonnes Pratiques Prompt Engineering:

1️⃣ Spécificité Scientifique:
   • Nommer explicitement tous les éléments attendus
   • Préciser les mesures, angles, proportions
   • Utiliser la terminologie technique correcte
   • Mentionner le niveau éducatif cible

2️⃣ Style Visuel:
   • 'textbook illustration' pour style académique
   • 'clean white background' pour clarté
   • 'clear labels' pour lisibilité
   • Spécifier les couleurs standard (CPK, etc.)

3️⃣ Pédagogie:
   • 'educational diagram' oriente vers usage scolaire
   • 'simplified but accurate' équilibre détail/clarté
   • Demander annotations et légendes
   • Inclure échelles et mesures

4️⃣ Qualité Technique:
   • Utiliser 'hd' pour diagrammes détaillés
   • Format 1792x1024 optimal p